In [129]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from conda.exports import download
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
import math

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
cuda = True if torch.cuda.is_available() else False

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

torch.manual_seed(125)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(125)



In [130]:
import torchvision.transforms as transforms

mnist_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (1.0,))
])

In [131]:
from torchvision.datasets import MNIST

download_root = '../chap07/MNIST_DATASET'

train_dataset = MNIST(download_root, transform=mnist_transform, train=True, download=True)
valid_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)
test_dataset = MNIST(download_root, transform=mnist_transform, train=False, download=True)

In [132]:
batch_size = 64
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)
valid_loader = DataLoader(dataset=test_dataset,
                          batch_size=batch_size,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=batch_size,
                          shuffle=True)

In [133]:
# images, labels = next(iter(train_loader))
#
# # Print the shape and other info
# print(f"Number of images in one batch: {images.shape[0]}")
# print(f"Image tensor shape: {images.shape}")
# print(f"Label tensor shape: {labels.shape}")
# print(f"Data type of image tensor: {images.dtype}")
# print(f"Data type of label tensor: {labels.dtype}")
# print(f"Sample labels: {labels[:5]}")

In [134]:
batch_size = 100
n_iters = 6000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

In [135]:
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, bias=True):
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 4 * hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 4 * hidden_size, bias=bias)
        self.reset_parameters()

    def reset_parameters(self):
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std, std)

    def forward(self, x, hidden):
        hx, cx = hidden
        x = x.view(-1, x.size(1))

        # W x [h_t-1, x_t] + b에 대응
        gates = self.x2h(x) + self.h2h(hx)
        # removes any dimensions of a tensor that have a size of 1
        # parameter로 n 넣어주면 n차원인 차원을 없애줘, 차원을 줄여
        # In neural network processing, data is often kept in batches, even if the batch size is just one. This line is a utility to flatten the tensor by removing any unnecessary dimensions, which can sometimes simplify subsequent calculations.
        gates = gates.squeeze()
        # 하나의 Linear layer로 계산 했던거 분리시켜
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1)

        ingate = F.sigmoid(ingate)
        forgetgate = F.sigmoid(forgetgate)
        cellgate = F.tanh(cellgate)
        outgate = F.sigmoid(outgate)

        # torch.mul(): element-wise multiplication 적용
        cy = torch.mul(cx, forgetgate) + torch.mul(ingate, cellgate)
        hy = torch.mul(outgate, F.tanh(cy))
        return (hy, cy)

In [136]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim  # 은닉층의 뉴런/유닛 갯수

        self.layer_dim = layer_dim
        self.lstm = LSTMCell(input_dim, hidden_dim, layer_dim)  # layer_dim looks total nonsense to me
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # hidden 값과 cell 값을 초기화
        # (은닉층의 계층 개수, 배치 크기, 은닉층의 뉴런 개수)
        if torch.cuda.is_available():
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda()
        else:
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)

        if torch.cuda.is_available():
            c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda()
        else:
            c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)

        outs = []
        # list slicing, [1번째 차원의 0번째 원소, 2번째 차원 전부다, 3번째 차원 전부다]
        cn = c0[0, :, :]
        hn = h0[0, :, :]

        for seq in range(x.size(1)):  # input tensor x: (batch_size, sequence_length, input_features)
            # In PyTorch, when you create a class that inherits from nn.Module and then call an instance of it like a function, you are automatically invoking its forward method.
            hn, cn = self.lstm(x[:, seq, :], (hn, cn)) # 2d tensor [batch_size, actual_input], memories for( hidden state, cell state))
            outs.append(hn)

        out = outs[-1].squeeze()  # 최종 결과를 1차원 배열로 squeze (b/c fc로 넘길거 잖아)
        out = self.fc(out)
        return out

In [137]:
input_dim = 28
hidden_dim = 128
layer_dim = 1
output_dim = 10

model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
if torch.cuda.is_available():
    model.cuda()
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimzer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [138]:
seq_dim = 28
loss_list = []
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = images.view(-1, seq_dim, input_dim).cuda()
            labels = labels.cuda()
        else:
            images = images.view(-1, seq_dim, input_dim)
            labels = labels

        optimzer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        if torch.cuda.is_available():
            loss.cuda()

        loss.backward()
        optimzer.step()
        loss_list.append(loss.item())
        iter += 1

        if iter % 500 == 0:
            correct = 0
            total = 0
            for images, labels in valid_loader:
                if torch.cuda.is_available():
                    images = images.view(-1, seq_dim, input_dim).cuda()
                else:
                    images = images.view(-1, seq_dim, input_dim)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                total += labels.size(0)
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()

            accuracy = 100 * correct / total
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 2.237457513809204. Accuracy: 21.420000076293945
Iteration: 1000. Loss: 0.9100692868232727. Accuracy: 74.06999969482422
Iteration: 1500. Loss: 0.4141859710216522. Accuracy: 87.98999786376953
Iteration: 2000. Loss: 0.31473594903945923. Accuracy: 91.62999725341797
Iteration: 2500. Loss: 0.09829992055892944. Accuracy: 93.63999938964844
Iteration: 3000. Loss: 0.07892906665802002. Accuracy: 96.4800033569336
Iteration: 3500. Loss: 0.09954178333282471. Accuracy: 96.30999755859375
Iteration: 4000. Loss: 0.013582933694124222. Accuracy: 97.12000274658203
Iteration: 4500. Loss: 0.05706917867064476. Accuracy: 97.12999725341797
Iteration: 5000. Loss: 0.08537448197603226. Accuracy: 97.27999877929688
Iteration: 5500. Loss: 0.0972917452454567. Accuracy: 97.05000305175781
Iteration: 6000. Loss: 0.026007307693362236. Accuracy: 97.81999969482422
Iteration: 6500. Loss: 0.021794160827994347. Accuracy: 97.62999725341797
Iteration: 7000. Loss: 0.06000397726893425. Accuracy: 97.8600006103

In [139]:
def evaluate(model, val_iter):
    corrects, total, total_loss = 0, 0, 0
    model.eval()
    for images, labels in val_iter:
        if torch.cuda.is_available():
            images = images.view(-1, seq_dim, input_dim).cuda()
        else:
            images = images.view(-1, seq_dim, input_dim).to(device)

        logit = model(images).to(device)
        loss = F.cross_entropy(logit, labels, reduction="sum")
        _, predicted = torch.max(logit.data, 1)
        total += labels.size(0)
        total_loss += loss.item()
        corrects += (predicted == labels).sum()

    avg_loss = total_loss / len(val_iter.dataset)
    avg_accuracy = corrects / total
    return avg_loss, avg_accuracy

test_loss, test_acc = evaluate(model, test_loader)
print("Test Loss: %5.2f | Test Accuracy: %5.2f" % (test_loss, test_acc))

Test Loss:  0.06 | Test Accuracy:  0.98
